In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from seirsd import SEIRSD
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import truncnorm

seirsd = SEIRSD()

# Valores de perda de imunidade: Sem perda, 1 ano, 6, 4, 3, 1 meses, 14 e 1 dias
# alfa_values = [0, 1/365, 1/180, 1/90, 1/30, 1/14, 1]
alfa_values = [0, 0.0027, 0.0056, 0.0111, 0.0333, 0.0714, 1]

beta = seirsd.get_default("beta")
sigma = seirsd.get_default("sigma")
gamma = seirsd.get_default("gamma")
mu = seirsd.get_default("mu")

initial_conditions = [seirsd.get_default("S"), seirsd.get_default("E"), seirsd.get_default("I"), seirsd.get_default("R"), seirsd.get_default("D")]

days = seirsd.get_default("days")
timespan_days = np.linspace(0, days, days+1)

In [2]:
results = {}


for alfa in alfa_values:
    rates = (beta, sigma, gamma, alfa, mu)
    sol = odeint(seirsd.odes, initial_conditions, timespan_days, args=(rates,))
    S, E, I, R, D = sol.T
    results[alfa] = {
        "t": timespan_days,
        "S": S,
        "E": E,
        "I": I,
        "R": R,
        "D": D,
        "total_deaths": D[-1]
    }

fig1 = go.Figure()

for alfa, data in results.items():
    label = f"α = {alfa:.4f} por dia"
    fig1.add_trace(go.Scatter(
        x=data["t"], 
        y=data["D"],
        mode='lines',
        name=label
    ))

fig1.update_layout(
    title="Impacto da perda de imunidade na mortalidade (Modelo SEIRSD)",
    xaxis_title="Tempo (dias)",
    yaxis_title="Mortos acumulados",
    legend_title="Taxa α de perda de imunidade",
)

fig1.show()

omegas_txt = [f"{w:.4f}" for w in alfa_values]
final_deaths = [results[w]["total_deaths"] for w in alfa_values]

fig2 = px.bar(
    x=omegas_txt,
    y=final_deaths,
    labels={'x': 'Taxa de perda de imunidade (por dia)', 'y': 'Mortos acumulados'},
    title="Mortalidade final por taxa de perda de imunidade",
)

fig2.show()



In [3]:
def sample_truncnorm(mean, sd, lower, upper, size):
    a, b = (lower - mean) / sd, (upper - mean) / sd
    return truncnorm.rvs(a, b, loc=mean, scale=sd, size=size, random_state=42)

N_sim = 1000  # por valor de alfa

cv = 0.1
results = {}

for alpha in alfa_values:
    D_curves = []
    D_final = []
    
    sigma_samples = sample_truncnorm(sigma, sigma*cv, sigma*0.5, sigma*1.5, N_sim)
    gamma_samples = sample_truncnorm(gamma, gamma*cv, gamma*0.5, gamma*1.5, N_sim)
    mu_samples = sample_truncnorm(mu, mu*cv, mu*0.5, mu*1.5, N_sim)
    beta_samples = sample_truncnorm(beta, beta*cv, beta*0.5, beta*1.5, N_sim)
    
    for i in range(N_sim):
        sigma = sigma_samples[i]
        gamma = gamma_samples[i]
        mu = mu_samples[i]
        beta = beta_samples[i]
        rates = (beta, sigma, gamma, alpha, mu)

        sol = odeint(seirsd.odes, initial_conditions, timespan_days, args=(rates, ))
        D_t = sol[:, 4]
        
        D_curves.append(D_t)
        D_final.append(D_t[-1])
    
    D_curves = np.array(D_curves)
    D_final = np.array(D_final)
    
    mean_curve = np.mean(D_curves, axis=0)
    low_curve = np.percentile(D_curves, 2.5, axis=0)
    high_curve = np.percentile(D_curves, 97.5, axis=0)
    
    results[alpha] = {
        "mean": mean_curve,
        "low": low_curve,
        "high": high_curve,
        "final_mean": np.mean(D_final),
        "final_low": np.percentile(D_final, 2.5),
        "final_high": np.percentile(D_final, 97.5)
    }

fig1 = go.Figure()
for alpha in alfa_values:
    res = results[alpha]
    fig1.add_trace(go.Scatter(
        x=timespan_days, y=res["mean"],
        mode='lines',
        name=f"α = {alpha:.4f} (média)"
    ))
    fig1.add_trace(go.Scatter(
        x=np.concatenate([timespan_days, timespan_days[::-1]]),
        y=np.concatenate([res["high"], res["low"][::-1]]),
        fill='toself',
        fillcolor='rgba(200,100,100,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=False
    ))

fig1.update_layout(
    title="Impacto da perda de imunidade na mortalidade (Monte Carlo)",
    xaxis_title="Tempo (dias)",
    yaxis_title="Mortos acumulados",
    template="plotly_white"
)

fig2 = go.Figure()
alphas_txt = [f"{a:.4f}" for a in alfa_values]
final_means = [results[a]["final_mean"] for a in alfa_values]
final_lows = [results[a]["final_low"] for a in alfa_values]
final_highs = [results[a]["final_high"] for a in alfa_values]

fig2.add_trace(go.Bar(
    x=alphas_txt, y=final_means,
    error_y=dict(type='data', array=np.array(final_highs)-np.array(final_means),
                 arrayminus=np.array(final_means)-np.array(final_lows)),
    marker_color='salmon'
))

fig2.update_layout(
    title="Mortalidade final por taxa de perda de imunidade (IC 95%)",
    xaxis_title="Taxa de perda de imunidade α (por dia)",
    yaxis_title="Mortos acumulados ao final (1 ano)",
    template="plotly_white"
)

fig1.show()
fig2.show()
